In [ ]:
import itertools

In [ ]:
from skyscanner import FlightsCache

service = FlightsCache('se893794935794863942245517499220')

params = dict(
    market='US',
    currency='USD',
    locale='en-US',
    destinationplace='US',
    outbounddate='2016-08',
    inbounddate='2016-08')

user1_params = dict(originplace='DTW-sky')
user2_params = dict(originplace='SFO-sky')

In [ ]:
user1_result = service.get_cheapest_quotes(**params, **user1_params).parsed
user2_result = service.get_cheapest_quotes(**params, **user2_params).parsed

In [ ]:
import sqlite3

DB_SCHEMA = """
PRAGMA foreign_keys = ON;

CREATE TABLE place
  ( id     INTEGER          PRIMARY KEY
  , name   TEXT    NOT NULL
  , type   TEXT    NOT NULL
  );

CREATE TABLE quote
  ( query_id                   INTEGER
  , quote_id                   INTEGER
  , direct                     BOOLEAN  NOT NULL
  , minimum_price              FLOAT    NOT NULL
  , quote_datetime             DATETIME NOT NULL
  , outbound_departure_date    DATETIME NOT NULL
  , outbound_origin_id         INTEGER  NOT NULL REFERENCES place(id)
  , outbound_destination_id    INTEGER  NOT NULL REFERENCES place(id)
  , outbound_carriers          TEXT     NOT NULL
  , inbound_departure_date     DATETIME NOT NULL
  , inbound_origin_id          INTEGER  NOT NULL REFERENCES place(id)
  , inbound_destination_id     INTEGER  NOT NULL REFERENCES place(id)
  , inbound_carriers           TEXT     NOT NULL
  
  , PRIMARY KEY (query_id, quote_id)
  );

"""

sqlite3.enable_callback_tracebacks(True)

db = sqlite3.connect(':memory:')
db.executescript(DB_SCHEMA)

places = [(d['PlaceId'], d['Name'], d['Type'])
          for d in itertools.chain(user1_result['Places'],
                                   user2_result['Places'])]
db.executemany('INSERT INTO place VALUES (?,?,?)', set(places))

quotes1 = [(1, d['QuoteId'], d['Direct'], d['MinPrice'], d['QuoteDateTime'],
           d['OutboundLeg']['DepartureDate'], d['OutboundLeg']['OriginId'],
           d['OutboundLeg']['DestinationId'], str(d['OutboundLeg']['CarrierIds']),
           d['InboundLeg']['DepartureDate'], d['InboundLeg']['OriginId'],
           d['InboundLeg']['DestinationId'], str(d['InboundLeg']['CarrierIds']))
          for d in user1_result['Quotes']]
db.executemany('INSERT INTO quote VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', quotes1)

quotes2 = [(2, d['QuoteId'], d['Direct'], d['MinPrice'], d['QuoteDateTime'],
           d['OutboundLeg']['DepartureDate'], d['OutboundLeg']['OriginId'],
           d['OutboundLeg']['DestinationId'], str(d['OutboundLeg']['CarrierIds']),
           d['InboundLeg']['DepartureDate'], d['InboundLeg']['OriginId'],
           d['InboundLeg']['DestinationId'], str(d['InboundLeg']['CarrierIds']))
          for d in user2_result['Quotes']]
db.executemany('INSERT INTO quote VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', quotes2)

In [ ]:
import pandas as pd

place_table = pd.read_sql('SELECT * FROM place', con=db)
place_map = dict(zip(place_table.id, place_table.name))

In [ ]:
query = """
    SELECT
        quote1.minimum_price + quote2.minimum_price AS minimum_total_price
      , quote1.outbound_origin_id AS quote1_origin
      , quote2.outbound_origin_id AS quote2_origin
      , quote1.outbound_destination_id AS destination
      , quote1.outbound_departure_date AS quote1_outbound_date
      , quote2.outbound_departure_date AS quote2_outbound_date
      , quote1.inbound_departure_date AS quote1_inbound_date
      , quote2.inbound_departure_date AS quote2_inbound_date
    FROM (SELECT * FROM quote where query_id = 1) AS quote1
    JOIN (SELECT * FROM quote where query_id = 2) AS quote2
      ON quote1.outbound_destination_id = quote2.outbound_destination_id
--      AND JULIANDAY(quote1.outbound_departure_date) = JULIANDAY(quote2.outbound_departure_date)
"""

data = pd.read_sql(query, con=db, parse_dates=['quote1_outbound_date', 'quote2_outbound_date', 'quote1_inbound_date', 'quote2_inbound_date'])
data.quote1_origin = data.quote1_origin.map(place_map)
data.quote2_origin = data.quote2_origin.map(place_map)
data.destination = data.destination.map(place_map)

In [ ]:
data[(abs((data.quote1_inbound_date - data.quote2_inbound_date).dt.days) <= 0) &
     (abs((data.quote1_outbound_date - data.quote2_outbound_date).dt.days) <= 0)].sort_values('minimum_total_price').head()